In [1]:
import datajoint as dj
from minio import Minio
from os import path

In [2]:
schema = dj.schema('common_distributed')

Connecting dimitri@ucsd-demo-db.datajoint.io:3306


In [3]:
bucket_name = 'mousebraindata-open'

def get_client():
    return Minio('s3.amazonaws.com',
                 access_key='???????',
                 secret_key='?????',
                 secure=True)

In [4]:
@schema
class BaseImage(dj.Manual):
    definition = '''
    image_id:          int            # image id
    ---
    image_location:    varchar(1024)  # image location in bucket
    '''
    
    @classmethod
    def fill(cls):   
        """
        fill the image list from the bucket
        """
        client = get_client()
        objects = client.list_objects(bucket_name, prefix='MD657/')
        cls.insert(
            dict(image_id=i, image_location=obj.object_name) 
            for i, obj in enumerate(objects) if obj.object_name.endswith('.jp2'))

In [ ]:
BaseImage.fill()

In [5]:
BaseImage()

image_id image id,image_location image location in bucket
3,MD657/MD657-F1-2017.02.17-17.39.26_MD657_1_0001_lossless.jp2
6,MD657/MD657-F1-2017.02.17-17.39.26_MD657_1_0001_lossy.jp2
9,MD657/MD657-F1-2017.02.17-17.39.26_MD657_2_0002_lossless.jp2
12,MD657/MD657-F1-2017.02.17-17.39.26_MD657_2_0002_lossy.jp2
15,MD657/MD657-F1-2017.02.17-17.39.26_MD657_3_0003_lossless.jp2
18,MD657/MD657-F1-2017.02.17-17.39.26_MD657_3_0003_lossy.jp2
21,MD657/MD657-F10-2017.02.17-19.35.42_MD657_1_0028_lossless.jp2


In [15]:
@schema
class Contrast(dj.Imported):
    definition = '''
    -> BaseImage
    ---
    contrast: float
    '''
    
    def make(self, key):
        key = ProcessedImage.key_source.fetch('KEY')[3]
        self = ProcessedImage()
        path = (BaseImage & key).fetch1('image_location')
        client = get_client()
        data = client.get_object(bucket_name, path).data
        img = convert_binary_jp2_data_into_image(data) 
        self.insert1(dict(key, contrast=img.std()))
        


In [ ]:
# distributed processing
Contrast.populate(reserve_jobs=True)